In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Oct_weekday_PM

No dicts found; please check load_dicts...


In [2]:
xi_list = GLS_Oct_weekday_PM.xi_list
P = GLS_Oct_weekday_PM.P
L = GLS_Oct_weekday_PM.L  # dimension of xi

314

In [3]:
size(P, 1), size(P, 2)

(56,314)

In [4]:
using JuMP

mGLSJulia = Model()

@defVar(mGLSJulia, lam[1:size(P,1)] >= 0)

@defVar(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @addConstraint(mGLSJulia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @addNLConstraint(mGLSJulia, sum{p[i,j], j = 1:size(P,2)} == 1)
end

for l = 1:L
    @addNLConstraint(mGLSJulia, sum{p[i,l] * lam[i], i = 1:size(P,1)} == xi_list[l])
end
    
@setNLObjective(mGLSJulia, Min, sum{p[1,j], j = 1:size(P,2)})  # play no actual role, but could not use zero objective

solve(mGLSJulia)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    70022
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    52752

Total number of variables............................:    17640
                     variables with only lower bounds:    17640
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [5]:
getValue(lam)

56-element Array{Float64,1}:
  534.318
  367.89 
  544.57 
  612.962
  551.11 
  635.075
  207.75 
  366.115
  400.606
  374.559
  575.626
  676.724
  674.191
    ⋮    
  873.587
  532.808
  713.648
 2405.39 
  219.054
  263.526
  293.03 
  677.3  
  418.235
  508.941
  742.177
  314.034

In [6]:
getObjectiveValue(mGLSJulia)

1.0000000000003229

In [7]:
GLS_Oct_weekday_PM.saveDemandVec(getValue(lam))